In [1]:
#PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import os
import sys

import cv2
from skimage import io, transform
import numpy as np

In [2]:
#PyTorch Models
path = os.path.join(os.path.dirname(os.path.abspath('__file__')), "models")
sys.path.append(path)
from models import *

In [3]:
model = ResNet152()
if torch.cuda.is_available():
    model = model.cuda()
    
path = "/home/bilal/ss18_hand/src/roboy_hand/gesture_recognition/saved_models/resnet_rgb_100_2018-07-04_06-59-19.model"
model.load_state_dict(torch.load(path))

In [4]:
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (0, 50)
fontScale              = 0.4
fontColor              = (255, 255, 255)
lineType               = 1

In [5]:
cv2.startWindowThread()
cam = cv2.VideoCapture(0)
while True:
    ret_val, img_org = cam.read()
    if ret_val != -1:
        img = transform.resize(img_org, (32, 32))

        img = np.moveaxis(img, [0, 1, 2], [1, 2, 0])

        img = torch.from_numpy(img).float()

        if torch.cuda.is_available():
            img = img.cuda()
        outputs = model.forward(img.unsqueeze_(0))
        
        _, preds = torch.max(outputs, 1)
        
        #str(outputs.data.cpu().numpy()[0])
        cv2.putText(img_org, str(preds.data.cpu().numpy()[0]), bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
        cv2.imshow("Image", img_org)
        #print(outputs)
    if cv2.waitKey(1) == 27:
        cam.release()
        break  # esc to quit
cv2.destroyAllWindows()

/home/bilal/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
